# Enrutamiento en FastAPI

El enrutamiento es una parte esencial de la construcción de una aplicación web. El enrutamiento en FastAPI es flexible y sin complicaciones. El enrutamiento es el proceso de manejar las **solicitudes** **HTTP** enviadas desde un cliente al servidor. Las solicitudes HTTP se envían a rutas definidas, que tienen manejadores definidos para procesar las solicitudes y responder. Estos manejadores se llaman manejadores de ruta.

Al final de este notebook, sabrás cómo crear rutas utilizando la instancia de ``APIRouter`` y conectarla a la aplicación principal de **FastAPI**. También aprenderás qué son los modelos y cómo usarlos para validar los cuerpos de las solicitudes. También aprenderás qué son los parámetros de ruta y de consulta y cómo usarlos en tu aplicación FastAPI. El conocimiento del enrutamiento en FastAPI es esencial para construir aplicaciones de pequeña y gran escala.
En este notebook, cubriremos los siguientes temas:

-  Enrutamiento en FastAPI
-  La clase APIRouter
-  Validación utilizando modelos Pydantic
-  Parámetros de ruta y de consulta
-  Cuerpo de la solicitud
-  Construcción de una sencilla aplicación CRUD

En términos más simples:

El enrutamiento es crucial cuando construyes una aplicación web. En FastAPI, es sencillo y flexible. Se trata de cómo se manejan las solicitudes HTTP de un cliente al servidor. Al final de este notebook, podrás crear rutas, validar datos, y usar parámetros en tu aplicación FastAPI. Estas habilidades son fundamentales para construir cualquier tipo de aplicación. Cubriremos el enrutamiento, la clase APIRouter, la validación con modelos Pydantic, parámetros de ruta y de consulta, y cómo construir una aplicación CRUD sencilla.

## Comprendiendo el enrutamiento en FastAPI

Una ruta se define para aceptar solicitudes de un método de solicitud HTTP y opcionalmente tomar parámetros. Cuando se envía una solicitud a una ruta, la aplicación verifica si la ruta está definida antes de procesar la solicitud en el manejador de rutas. Por otro lado, un manejador de rutas es una función que procesa la solicitud enviada al servidor. Un ejemplo de un manejador de rutas es una función que recupera registros de una base de datos cuando se envía una solicitud a un enrutador a través de una ruta.

Ejemplo de enrutamiento

En la sección de estructura del proyecto en el notebook anterior, construimos una aplicación con una sola ruta. El enrutamiento fue manejado por la instancia FastAPI() iniciada en la variable app:

In [ ]:
from fastapi import FastAPI
app = FastAPI()

@app.get("/")
async def welcome() -> dict:
    return { "message": "Hello World"}

La herramienta uvicorn se dirigió a la instancia de FastAPI para servir la aplicación:

> uvicorn app:app --port 8080 --reload

Tradicionalmente, la instancia FastAPI() se puede usar para operaciones de enrutamiento, como se vio anteriormente. Sin embargo, este método se utiliza comúnmente en aplicaciones que requieren un solo camino durante el enrutamiento. En una situación en la que se crea una ruta separada que realiza una función única usando la instancia FastAPI(), la aplicación no podrá ejecutar ambas rutas, ya que uvicorn solo puede ejecutar un punto de entrada.

¿Cómo manejas entonces aplicaciones extensas que requieren una serie de rutas que realizan diferentes funciones? Veremos cómo la clase ``APIRouter`` ayuda con múltiples enrutamientos en la siguiente sección.

## Enrutamiento con la clase APIRouter

La clase APIRouter pertenece al paquete FastAPI y crea operaciones de ruta para múltiples rutas. La clase APIRouter fomenta la modularidad y organización del enrutamiento de aplicaciones y la lógica.

La clase APIRouter se importa del paquete fastapi y se crea una instancia. Los métodos de ruta se crean y distribuyen desde la instancia creada, de la siguiente manera:

In [ ]:
from fastapi import APIRouter
router = APIRouter()

@router.get("/hello")
async def say_hello() -> dict:
    return {"message": "Hello!"}

Vamos a crear una nueva operación de ruta con la clase APIRouter para crear y recuperar "todos". En la carpeta "todos" del notebook anterior, crea un nuevo archivo, [todo.py](../app/todos/todo.py):

Comenzaremos importando la clase APIRouter del paquete fastapi y creando una instancia:

In [ ]:
from fastapi import APIRouter
todo_router = APIRouter()

A continuación, crearemos una base de datos temporal en la aplicación, junto con dos rutas para la adición y recuperación de "todos":

In [ ]:
todo_list = []

@todo_router.post("/todo")
async def add_todo(todo: dict) -> dict:
    todo_list.append(todo)
    return {"message": "Todo added successfully"}

@todo_router.get("/todo")
async def retrieve_todos() -> dict:
    return {"todos": todo_list}


En el bloque de código anterior, hemos creado dos rutas para nuestras operaciones de "todo". La primera ruta agrega un "todo" a la lista de "todo" a través del método POST, y la segunda ruta recupera todos los elementos "todo" de la lista de "todo" a través del método GET.

Hemos completado las operaciones de ruta para la ruta "todo". El siguiente paso es ejecutar la aplicación a producción para que podamos probar las operaciones de ruta definidas.

La clase APIRouter funciona de la misma manera que la clase FastAPI. Sin embargo, uvicorn no puede usar la instancia de APIRouter para servir la aplicación, a diferencia de las FastAPIs. Las rutas definidas usando la clase APIRouter se añaden a la instancia de fastapi para permitir su visibilidad.

Para permitir la visibilidad de las rutas de "todo", incluiremos el manejador de operaciones de ruta "todo_router" a la instancia FastAPI principal utilizando el método include_router().

En "app.py", importa "todo_router" desde "todo.py":

In [ ]:
from todo import todo_router

Incluye el "todo_router" en la aplicación FastAPI, utilizando el método "include_router" de la instancia FastAPI:

In [ ]:
from fastapi import FastAPI
from todo import todo_router

app = FastAPI()

@app.get("/")
async def welcome() -> dict:
    return {
    "message": "Hello World"
    }

app.include_router(todo_router)

Con todo en su lugar, inicia la aplicación desde tu terminal:

> uvicorn app:app --port 8080 --reload

```
Para probarlo:

- Homepage "/":
curl http://localhost:8080

- Get "/todo":
curl -X GET "http://localhost:8080/todo" -H "accept: application/json"

- Post "/todo":
curl -X POST "http://localhost:8080/todo" -H "accept: application/json" -H "Content-Type: application/json" -d "{\"id\": 1, \"item\": \"¡Mi primera tarea es aprender FastAPI!\"}"

```

## Validando cuerpos de solicitud usando modelos Pydantic

En FastAPI, los cuerpos de las solicitudes pueden ser validados para asegurar que sólo se envíen los datos definidos. Esto es crucial, ya que sirve para sanear los datos de la solicitud y reducir los riesgos de ataques maliciosos. A este proceso se le conoce como validación.

Un modelo en FastAPI es una clase estructurada que dicta cómo se deben recibir o analizar los datos. Los modelos se crean al subclasificar la clase BaseModel de Pydantic.

Cuando se definen, los modelos se utilizan como sugerencias de tipo para los objetos del cuerpo de la solicitud y los objetos de respuesta de la solicitud. En este notebook, sólo veremos el uso de los modelos Pydantic para los cuerpos de las solicitudes.

Un ejemplo de modelo es el siguiente:

In [ ]:
from pydantic import BaseModel
class PacktBook(BaseModel):
    id: int
    Name: str
    Publishers: str
    Isbn: str

En el bloque de código anterior, definimos un modelo PacktBook como una subclase de la clase BaseModel de Pydantic. Una variable con una sugerencia de tipo a la clase PacktBook sólo puede tener cuatro campos, como se definió anteriormente. En los siguientes ejemplos, veremos cómo Pydantic ayuda a validar las entradas.

En nuestra aplicación "todo" de antes, definimos una ruta para añadir un ítem a nuestra lista de "todos". En la definición de la ruta, establecimos el cuerpo de la solicitud en un diccionario:

In [ ]:
async def add_todo(todo: dict) -> dict:

En el ejemplo de solicitud POST, los datos enviados tenían el siguiente formato:

In [ ]:
{
"id": id,
"item": item
}

Sin embargo, también se podría haber enviado un diccionario vacío sin que se devuelva ningún error. Un usuario puede enviar una solicitud con un cuerpo distinto al mostrado anteriormente. Crear un modelo con la estructura de cuerpo de solicitud requerida y asignarlo como tipo al cuerpo de la solicitud asegura que sólo se pasan los campos de datos presentes en el modelo.

Por ejemplo, para asegurar que sólo el cuerpo de la solicitud contiene campos en el ejemplo anterior, crea un nuevo archivo [model.py](../app/todos/model.py) y añade el siguiente código a continuación:

In [ ]:
from pydantic import BaseModel
class Todo(BaseModel):
    id: int
    item: str

En el bloque de código anterior, hemos creado un modelo Pydantic que sólo acepta dos campos:

- id de tipo entero
- item de tipo string

Vamos a adelante y usemos el modelo en nuestra ruta POST. 

En [todo.py](../app/todos/todo.py), importa el modelo:

In [ ]:
from model import Todo

A continuación, reemplaza el tipo de variable del cuerpo de la solicitud de dict a Todo:

In [ ]:
todo_list = []

@todo_router.post("/todo")
async def add_todo(todo: Todo) -> dict:
    todo_list.append(todo)
    return {"message": "Todo added successfully"}

@todo_router.get("/todo")
async def retrieve_todos() -> dict:
    return {"todos": todo_list}

Ahora vamos a verificarlo:

```
curl -X POST "http://localhost:8080/todo" -H "accept: application/json" -H "Content-Type: application/json" -d "{}"
```

In [ ]:
{
    "detail": [
        {
            "loc": [
                "body",
                "id"
            ],
            "msg": "field required",
            "type": "value_error.missing"
            },
            {
                "loc": [
                "body",
                "item"
            ],
            "msg": "field required",
            "type": "value_error.missing"
        }
    ]
}

Ahora vamos a meter datos con el POST:

```
curl -X POST "http://localhost:8080/todo" -H "accept: application/json" -H "Content-Type: application/json" -d "{\"id\": 2, \"item\": \"Validar el tipo de datos con model.py\"}"
```

## Modelos Anidados
Los modelos Pydantic también pueden ser anidados, como el siguiente:

In [ ]:
class Item(BaseModel):
    item: str
    status: str
class Todo(BaseModel):
    id: int
    item: Item

Como resultado, un 'todo' de tipo Todo se representará de la siguiente manera:

In [ ]:
{
    "id": 1,
    "item": {
        "item": "Modelos anidados",
        "Status": "completed"
        }
    }

Hemos aprendido qué son los modelos, cómo crear uno y sus casos de uso. Los utilizaremos posteriormente en las partes restantes de este libro. En la siguiente sección, veamos los parámetros de ruta y de consulta.

```
curl -X POST "http://localhost:8080/todo" -H "accept: application/json" -H "Content-Type: application/json" -d "{\"id\": 3, \"item\": {\"item\": \"Cuanto falta para terminar\", \"status\": \"Cansad@\"}}"
```

## Parámetros de Ruta y Consulta
En la sección anterior, aprendimos qué son los modelos y cómo se utilizan para validar los cuerpos de las solicitudes. En esta sección, aprenderás qué son los parámetros de ruta y de consulta, el papel que juegan en el enrutamiento y cómo usarlos.

### Parámetros de Ruta
Los parámetros de ruta son parámetros incluidos en una ruta de API para identificar recursos. Estos parámetros sirven como un identificador y, a veces, un puente para permitir operaciones adicionales en una aplicación web.

Actualmente tenemos rutas para añadir un 'todo' y recuperar todos los 'todos' en nuestra aplicación. Vamos a crear una nueva ruta para recuperar un solo 'todo' agregando el ID del 'todo' como un parámetro de ruta.

En [todo.py](../app/todos/todo.py), añade la nueva ruta:

In [ ]:
from fastapi import APIRouter, Path
from model import Todo

todo_router = APIRouter()
todo_list = []

@todo_router.post("/todo")
async def add_todo(todo: Todo) -> dict:
    todo_list.append(todo)
    return {
    "message": "Todo added successfully."
    }

@todo_router.get("/todo")
async def retrieve_todo() -> dict:
    return {
    "todos": todo_list
    }

@todo_router.get("/todo/{todo_id}")
async def get_single_todo(todo_id: int = Path(..., title="The ID of the todo to retrieve.")) -> dict:
    for todo in todo_list:
        if todo.id == todo_id:
            return {
                "todo": todo
            }
    return {
        "message": "Todo with supplied ID doesn't exist."
    }

En el bloque de código anterior, {todo_id} es el parámetro de ruta. Este parámetro permite que la aplicación devuelva un 'todo' que coincida con el ID pasado.

Lo podemos comprobar de la siguente forma:

```
curl -X GET "http://localhost:8080/todo/1" -H "accept: application/json"
```

FastAPI también proporciona una clase Path que distingue los parámetros de ruta de otros argumentos presentes en la función de ruta. La clase Path también ayuda a dar a los parámetros de ruta más contexto durante la documentación proporcionada automáticamente por OpenAPI a través de Swagger y ReDoc y actúa como un validador.

Modifiquemos la definición de la ruta:

In [ ]:
from fastapi import APIRouter, Path
from model import Todo

todo_router = APIRouter()

todo_list = []

@todo_router.post("/todo")
async def add_todo(todo: Todo) -> dict:
    todo_list.append(todo)
    return {
        "message": "Todo added successfully."
    }

@todo_router.get("/todo")
async def retrieve_todo() -> dict:
    return {
        "todos": todo_list
    }

@todo_router.get("/todo/{todo_id}")
async def get_single_todo(todo_id: int = Path(..., title="The ID of the todo to retrieve")) -> dict:
    for todo in todo_list:
        if todo.id == todo_id:
            return {
                "todo": todo
                }
        return {
            "message": "Todo with supplied ID doesn't exist."
}

### Parámetros de Consulta
Un parámetro de consulta es un parámetro opcional que suele aparecer después de un signo de interrogación en una URL. Se utiliza para filtrar solicitudes y devolver datos específicos en función de las consultas suministradas.

En una función de controlador de ruta, un argumento que no es homónimo con el parámetro de ruta es una consulta. También puedes definir una consulta creando una instancia de la clase Query() de FastAPI en el argumento de la función, como el siguiente:

In [ ]:
async def query_route(query: str = Query(None)):
    return query

Examinaremos los casos de uso de los parámetros de consulta más adelante en la Clase cuando discutamos cómo construir aplicaciones más avanzadas que una aplicación de 'todo'.

Ahora que has aprendido cómo crear rutas, validar los cuerpos de las solicitudes y utilizar los parámetros de ruta y de consulta en tu aplicación FastAPI, aprenderás cómo estos componentes trabajan conjuntamente para formar un cuerpo de solicitud en la siguiente sección.

## Cuerpo de la Solicitud

En las secciones anteriores, aprendimos cómo usar la clase APIRouter y los modelos Pydantic para las validaciones del cuerpo de la solicitud y discutimos los parámetros de ruta y de consulta.

Un cuerpo de solicitud es datos que envías a tu API utilizando un método de enrutamiento como POST y UPDATE.

Hemos aprendido sobre los modelos en FastAPI. También sirven un propósito adicional en la documentación de nuestros puntos finales de la API y tipos de cuerpo de solicitud. En la siguiente subsección, aprenderemos sobre las páginas de documentación generadas por defecto en las aplicaciones FastAPI.

```
curl -X POST "http://localhost:8080/todo" -H "accept: application/json" -H "Content-Type: application/json" -d "{\"id\": 2, \"item\": \"Validar models con los inputs\"}"
```

<div>
    <a href="Intro.py">
        <img src="img/return.png" alt="return" title="return" width="75" style="float: left;" />
    </a>
    <a href="./NB03.ipynb">
        <img src="img/forward.png" alt="forward" title="forward" width="75" style="float: right;" />
    </a>
</div>